## Analyze whether SNWD varies more from year to year or from place to place.

In [1]:
import pandas as pd
import numpy as np
import urllib
import math

In [2]:
import findspark
findspark.init()

from pyspark import SparkContext
#sc.stop()
sc = SparkContext(master="local[3]",pyFiles=['lib/numpy_pack.py','lib/spark_PCA.py','lib/computeStats.py'])

from pyspark import SparkContext
from pyspark.sql import *
sqlContext = SQLContext(sc)

In [3]:
import sys
sys.path.append('./lib')

import numpy as np
from numpy_pack import packArray,unpackArray
from spark_PCA import computeCov
from computeStats import computeOverAllDist, STAT_Descriptions

In [4]:
### Read the data frame from pickle file

data_dir='../../Data/Weather'
#file_index='BBBSBBBB'
file_index='SSSSBSBB'
meas='TOBS'

from pickle import load

#read statistics
filename=data_dir+'/STAT_%s.pickle'%file_index
STAT,STAT_Descriptions = load(open(filename,'rb'))
print('keys from STAT=',STAT.keys())

#!ls -ld $data_dir/*.parquet

#read data
filename=data_dir+'/decon_%s_%s.parquet'%(file_index,meas)

df=sqlContext.read.parquet(filename)
print(df.count())
df.show(2)

('keys from STAT=', ['TMIN', 'TOBS', 'TMAX', 'SNOW', 'SNWD', 'PRCP'])
1543
+------------------+------------------+-------------------+---------+--------+--------+---------+-----------+------------------+------------------+------------------+-------------------+-----------+---------+------+--------------------+------+
|           coeff_1|           coeff_2|            coeff_3|elevation|   label|latitude|longitude|measurement|             res_1|             res_2|             res_3|           res_mean|    station|total_var|undefs|              vector|  year|
+------------------+------------------+-------------------+---------+--------+--------+---------+-----------+------------------+------------------+------------------+-------------------+-----------+---------+------+--------------------+------+
|253.53137090050058|-7.906590483099212|-194.06608098351094|   2114.4|SSSSBSBB|   33.35|-105.6667|       TOBS|0.9254717138776013| 0.925399230868368|0.8817318648855269|0.08654085361231952|USC0029

In [5]:
print df.columns

['coeff_1', 'coeff_2', 'coeff_3', 'elevation', 'label', 'latitude', 'longitude', 'measurement', 'res_1', 'res_2', 'res_3', 'res_mean', 'station', 'total_var', 'undefs', 'vector', 'year']


In [6]:
#extract longitude and latitude for each station
feature='coeff_1'
sqlContext.registerDataFrameAsTable(df,'weather')
#Features=', '.join(['coeff_1', 'coeff_2', 'coeff_3', 'elevation', 'latitude', 'longitude',\
#          'res_1', 'res_2', 'res_3', 'res_mean', 'year'])
Features='station, year, coeff_1, coeff_2, coeff_3'
Query="SELECT %s FROM weather"%Features
print(Query)
pdf = sqlContext.sql(Query).toPandas()
pdf.head()

SELECT station, year, coeff_1, coeff_2, coeff_3 FROM weather


,station,year,coeff_1,coeff_2,coeff_3
0,USC00297651,1988.0,253.531371,-7.906590,-194.066081
1,USC00297651,1989.0,-51.350022,175.135368,-100.882811
2,USC00297651,1990.0,164.706102,-20.270314,-145.911197
3,USC00297651,1991.0,349.668629,179.558337,-233.533997
4,USC00297651,1992.0,451.716625,22.223982,3.315219


In [7]:
year_station_table=pdf.pivot(index='year', columns='station', values='coeff_2')
year_station_table.head(10)

station,USC00022754,USC00290199,USC00290200,USC00290268,USC00291286,USC00291901,USC00291910,USC00291927,USC00291931,USC00291933,...,USC00297340,USC00297651,USC00298095,USC00298324,USC00299128,USC00299165,USC00299686,USC00299691,USC00299806,USS0008T01S
year,,,,,,,,,,,,,,,,,,,,,
1901.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1902.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1903.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1904.0,260.100296,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1905.0,144.154613,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1906.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,80.566861,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1907.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,122.941118,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1908.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1909.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


### Estimating the effect of the year vs the effect of the station

To estimate the effect of time vs. location on the first eigenvector coefficient we
compute:

* The average row: `mean-by-station`
* The average column: `mean-by-year`

We then compute the RMS before and after subtracting either  the row or the column vector.

In [8]:
def RMS(Mat):
    return np.sqrt(np.nanmean(Mat**2))

year_station_table=pdf.pivot(index='year', columns='station', values='coeff_1')
mean_by_year=np.nanmean(year_station_table,axis=1)
mean_by_station=np.nanmean(year_station_table,axis=0)
tbl_minus_year = (year_station_table.transpose()-mean_by_year).transpose()
tbl_minus_station = year_station_table-mean_by_station

print 'coeff_1'
print 'total RMS                   = ',RMS(year_station_table)
print 'RMS removing mean-by-station= ',RMS(tbl_minus_station)
print 'fraction explained  =', (RMS(year_station_table) - RMS(tbl_minus_station)) / RMS(year_station_table) 
print 'RMS removing mean-by-year   = ',RMS(tbl_minus_year)
print 'fraction explained  =', (RMS(year_station_table) - RMS(tbl_minus_year)) / RMS(year_station_table) 

coeff_1
total RMS                   =  761.58485882
RMS removing mean-by-station=  512.026911995
fraction explained  = 0.32768239013
RMS removing mean-by-year   =  735.248888647
fraction explained  = 0.0345804802553


In [9]:
def RMS(Mat):
    return np.sqrt(np.nanmean(Mat**2))

year_station_table=pdf.pivot(index='year', columns='station', values='coeff_2')
mean_by_year=np.nanmean(year_station_table,axis=1)
mean_by_station=np.nanmean(year_station_table,axis=0)
tbl_minus_year = (year_station_table.transpose()-mean_by_year).transpose()
tbl_minus_station = year_station_table-mean_by_station

print 'coeff_2'
print 'total RMS                   = ',RMS(year_station_table)
print 'RMS removing mean-by-station= ',RMS(tbl_minus_station)
print 'fraction explained', (RMS(year_station_table) - RMS(tbl_minus_station)) / RMS(year_station_table) 
print 'RMS removing mean-by-year   = ',RMS(tbl_minus_year)
print 'fraction explained', (RMS(year_station_table) - RMS(tbl_minus_year)) / RMS(year_station_table) 

coeff_2
total RMS                   =  187.242044603
RMS removing mean-by-station=  181.193612257
fraction explained 0.0323027467443
RMS removing mean-by-year   =  125.66970905
fraction explained 0.328838192748


In [10]:
def RMS(Mat):
    return np.sqrt(np.nanmean(Mat**2))

year_station_table=pdf.pivot(index='year', columns='station', values='coeff_3')
mean_by_year=np.nanmean(year_station_table,axis=1)
mean_by_station=np.nanmean(year_station_table,axis=0)
tbl_minus_year = (year_station_table.transpose()-mean_by_year).transpose()
tbl_minus_station = year_station_table-mean_by_station

print 'coeff_3'
print 'total RMS                   = ',RMS(year_station_table)
print 'RMS removing mean-by-station= ',RMS(tbl_minus_station)
print 'fraction explained', (RMS(year_station_table) - RMS(tbl_minus_station)) / RMS(year_station_table) 
print 'RMS removing mean-by-year   = ',RMS(tbl_minus_year)
print 'fraction explained', (RMS(year_station_table) - RMS(tbl_minus_year)) / RMS(year_station_table) 

coeff_3
total RMS                   =  187.471225098
RMS removing mean-by-station=  164.778155903
fraction explained 0.121048279184
RMS removing mean-by-year   =  157.571572096
fraction explained 0.159489292215


In [11]:
T=year_station_table
print 'initial RMS=',RMS(T)
for i in range(5):
    mean_by_year=np.nanmean(T,axis=1)
    T=(T.transpose()-mean_by_year).transpose()
    print i,'after removing mean by year    =',RMS(T)
    mean_by_station=np.nanmean(T,axis=0)
    T=T-mean_by_station
    print i,'after removing mean by stations=',RMS(T)

initial RMS= 187.471225098
0 after removing mean by year    = 157.571572096
0 after removing mean by stations= 134.606926262
1 after removing mean by year    = 133.99709044
1 after removing mean by stations= 133.939602546
2 after removing mean by year    = 133.928586135
2 after removing mean by stations= 133.925499434
3 after removing mean by year    = 133.924435947
3 after removing mean by stations= 133.924019339
4 after removing mean by year    = 133.923842262
4 after removing mean by stations= 133.9237632
